In [2]:
import pandas as pd
import numpy as np
import os
import time

In [4]:
def Merges(list1, list2):
    
    a = pd.DataFrame(list1)
    b = pd.DataFrame(list2)
    
    c = pd.merge(a,b, on = a.index)
    c.columns = range(c[c.index==0].size)
    c.set_index(c[0])
    c=c.drop(0,1)
    
    return c

In [5]:
def FindXY(df,df_cz):
    
    c=pd.DataFrame()
    tmp = 0
    now = 0
    count=0
    start = time.time()
    for cid in df['콘존ID'][:]:

        tmp_df = df_cz[df_cz['콘존ID']==cid]
        if tmp_df.empty:
            c=c.append(pd.DataFrame({'콘존ID':[cid],"위도":[0],"경도":[0]}))
        else:
            c=c.append(tmp_df)

        now = int(count/len(df)*100)
        if(now>tmp):
            print('%3d%%' % now, end='', flush = True)
        tmp = now
        count+=1
    end = time.time()
    tt = int(end-start)
    print('')
    print('[완료] %d초 소요'%tt)
    print('len(c) : %d'%len(c))

    cols = df.columns.values
    cols = np.append(cols,c.columns.values[1:])
    
    tmp = Merges(df,c)
    
    c_drop = len(df.columns.values)+1
    tmp=tmp.drop(c_drop,1)
    
    tmp.columns=cols

    return tmp

In [38]:
folder='VDS_FAST'
path = r'%s'%folder

# 디렉토리의 전체 파일 리스트 저장
file_list= os.listdir(path)


# 정상/오류 파일 개수 
count = 0
error_count=0

#진행 상황
m_sum = 0 # 현재 완료된 파일 크기 저장
tmp = -1  # 중복된 % 제거
m_all = 0 # 파일 전체 크기 저장

d = len(file_list) # 몇개 돌릴지

excs = ['.ipynb_checkpoints'] # csv가 아닌 파일 제외

# 폴더의 전체 파일크기 파악
for i in file_list[:d]:
    
    strs = folder+'/'+i
    
    m_ctn = False
    for j in excs: 
        if(i==j):
            m_ctn=True
        else:
            m_ctn=False
    if(m_ctn):
        continue
    
    
    m_stat = os.stat(strs)
    m_size=m_stat.st_size
    m_sum+=m_size

m_all = m_sum
m_sum = 0

print(folder,' : %.2fMB' % (m_all/(1024*1024)))



start = time.time()

for i in file_list[:d]:
    
    strs = folder+'/'+i
    
    # 미리 설정한 파일 제외
    m_ctn = False
    for j in excs:
        if(i==j):
            m_ctn=True
        else:
            m_ctn=False
    if(m_ctn):
        continue
    
    try:
        df = pd.read_csv(strs, sep="|", encoding='CP949', error_bad_lines=False, warn_bad_lines=False)
        df.columns=['일자','시간','콘존ID','차로유형','통행속도','???']
        df=df.drop('???',1)
        a = str(df['일자'][0])
        df.to_csv(r'%s'%('CSV_FAST/fast_'+a[:6]+'.csv'))
    except:
        print(strs," 파일에서 에러발생")
        error_count+=1
    
    count+=1
        
    m_stat = os.stat(strs)
    m_size=m_stat.st_size
    m_sum+=m_size
    now = int(m_sum/m_all*100)
    if(now>tmp):
        print('%3d%%' % now, end='', flush = True)
    tmp = now
end = time.time()

print('\n소요시간 : %d초 (%d분)' % (int(end-start),(int(end-start)/60)))
print('총 파일 %d개' % count)
print('총 에러 %d개' % error_count)

print('len(df) : ',len(df))

#==> 소요시간 : 초
#==> 총 파일 1577개
#==> 총 에러 0개

VDS_FAST  : 134.62MB
100%
소요시간 : 34초 (0분)
총 파일 1개
총 에러 0개
len(df) :  3996591


'201808'

In [3]:
df2 = pd.read_csv('test_fast.csv',sep=",")
df2= df2.set_index('Unnamed: 0')
df2.head(5)

,일자,시간,콘존ID,차로유형,통행속도
Unnamed: 0,,,,,
0,20170715,805,0500CZS230,1,99.80
1,20170715,805,0500CZS240,1,103.12
2,20170715,805,0500CZS250,1,95.58
3,20170715,805,0500CZS260,1,101.83
4,20170715,805,0500CZS270,1,0.00


In [4]:
df3 = df2.drop_duplicates('콘존ID',keep='first')
len(df3)

1274

In [8]:
df_cz = pd.read_csv('acc_conzone.csv',sep=',',encoding='CP949')
df_cz = df_cz.drop('Unnamed: 0',1)
str_arr= ['콘존ID','위도','경도']
df_cz2 = df_cz[str_arr]

#중복제거
df_cz2 = df_cz2.drop_duplicates('콘존ID',keep='first')
print('위도,경도 표시되는 콘존ID 개수 : ',len(df_cz2))
print('샘플 데이터의 콘존ID 개수 : ',len(df3))

print('좌표 못찾는 콘존ID 개수 : ',len(df3)-len(df_cz2))
df_cz2.head(5)

위도,경도 표시되는 콘존ID 개수 :  1030
샘플 데이터의 콘존ID 개수 :  1274
좌표 못찾는 콘존ID 개수 :  244


,콘존ID,위도,경도
0,0010CZE010,35.256977,129.101614
1,0010CZE011,35.265211,129.105973
3,0010CZE020,35.283054,129.104303
4,0010CZE030,35.287171,129.100226
5,0010CZE035,35.293179,129.089296


In [9]:
df4 = FindXY(df3,df_cz2)
print(len(df4))
df4.head(5)

  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28% 29% 30% 31% 32% 33% 34% 35% 36% 37% 38% 39% 40% 41% 42% 43% 44% 45% 46% 47% 48% 49% 50% 51% 52% 53% 54% 55% 56% 57% 58% 59% 60% 61% 62% 63% 64% 65% 66% 67% 68% 69% 70% 71% 72% 73% 74% 75% 76% 77% 78% 79% 80% 81% 82% 83% 84% 85% 86% 87% 88% 89% 90% 91% 92% 93% 94% 95% 96% 97% 98% 99%
[완료] 2초 소요
len(c) : 1274
1274


,일자,시간,콘존ID,차로유형,통행속도,위도,경도
0,20170715,805,0500CZS230,1,99.80,37.691969,127.541312
1,20170715,805,0500CZS240,1,103.12,37.739504,127.638648
2,20170715,805,0500CZS250,1,95.58,37.749142,127.915491
3,20170715,805,0500CZS260,1,101.83,37.903127,128.273206
4,20170715,805,0500CZS270,1,0.00,37.977287,128.435622


In [10]:
df5= df4[df4['위도']==0]
df5.head(5)

,일자,시간,콘존ID,차로유형,통행속도,위도,경도
164,20170715,805,0651CZS030,1,106.62,0.0,0.0
165,20170715,805,0651CZS040,1,108.82,0.0,0.0
166,20170715,805,0651CZS050,1,114.55,0.0,0.0
167,20170715,805,0651CZS060,1,105.34,0.0,0.0
168,20170715,805,0651CZS070,1,96.41,0.0,0.0


In [17]:
df6=df5['콘존ID']
df6.index = range(len(df6))
df6.columns = ['콘존ID']
#df6.head(5)
df6.to_csv(r'no_xy_conzone.csv')

c:\programdata\miniconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """


In [44]:
no_xy = pd.read_csv('no_xy_conzone.csv',sep=',',encoding='CP949')
no_xy = no_xy.drop('0',1)
no_xy.columns = ["콘존ID"]
no_xy.head(5)

,콘존ID
0,0651CZS040
1,0651CZS050
2,0651CZS060
3,0651CZS070
4,0652CZE010


In [30]:
cz_gugan = pd.read_csv('czid_to_gugan.txt',sep="\t",encoding='CP949')

cols = cz_gugan.columns
cols = cols[0].split(',')

cz_gugan.index = range(len(cz_gugan))
cz_gugan.columns=range(1)

cz_gugan = cz_gugan[0].str.split(',',n=12,expand=True)
cz_gugan.columns = cols

cz_gugan.head(5)

1438


,콘존ID,콘존길이,기점종점방향구분코드,시작노드ID,종료노드ID,차로수,노선번호,제한속도,노선구성순번,콘존명,버스전용차로유무,도로등급구분코드
0,0010CZE005,200.00,E,0491,0004,3,0010,100.00,1,경부고속국도시점-구서IC,0,101
1,0010CZE010,1820.00,E,0004,0446,3,0010,100.00,2,구서IC-영락IC,0,101
2,0010CZE011,1990.00,E,0446,0486,3,0010,100.00,3,영락IC-부산TG,0,101
3,0010CZE020,1070.00,E,0486,0447,3,0010,100.00,4,부산TG-노포,0,101
4,0010CZE030,7780.00,E,0447,0669,3,0010,100.00,5,노포IC-노포JC,0,101


In [53]:
count=0
for n in no_xy['콘존ID']:
    m_cf = False
    for g in cz_gugan['콘존ID']:
        if n in g:
            count+=1
            m_cf = True
    if m_cf==False:
        print(n)
print('count : ',count)
print('len(no_xy) : ',len(no_xy))


0170CZE090
0170CZE100
0170CZS090
0170CZS100
count :  294
len(no_xy) :  298


In [4]:
## 통행량

#highway/data/VDS_COUNT
folder='VDS_COUNT'
path = r'%s'%folder

df1=pd.DataFrame(
file_list= os.listdir(path)

start = time.time()

# 정상/오류 파일 개수 
count = 0
error_count=0

#진행 상황
m_sum = 0
tmp = -1
m_all = 0

d=len(file_list)

for i in file_list[:d]:
    
    strs = folder+'/'+i
    m_stat = os.stat(strs)
    m_size=m_stat.st_size
    m_sum+=m_size

m_all = m_sum
m_sum = 0

print(folder,' : %.2fMB' % (m_all/(1024*1024)))

for i in file_list[:d]:

    strs = folder+'/'+i
    
    try:
        df_tmp = pd.read_csv(strs, sep="|", encoding='CP949', error_bad_lines=False, warn_bad_lines=False)
        df1=df1.append(df_tmp,sort=False)
        del df_tmp
    except:
        #print(strs," 파일에서 에러발생")
        error_count+=1
    
    count+=1
        
    m_stat = os.stat(strs)
    m_size=m_stat.st_size
    m_sum+=m_size
    now = int(m_sum/m_all*100)
    if(now>tmp):
        print('%3d%%' % now, end='', flush = True)
    tmp = now
    
end = time.time()

print('\n소요시간 : %d초 (%d분)' % (int(end-start),(int(end-start)/60)))
print('총 파일 %6d개' % count)
print('총 에러 %6d개' % error_count)

df1.columns=['일자','시간','콘존ID','차로유형','통행량','???']
df1=df1.drop('???',1)
df1.head(5)
#==> 소요시간 : 초
#==> 총 파일 1577개
#==> 총 에러 0개

SyntaxError: invalid syntax (<ipython-input-4-0af0a164c8e7>, line 12)